# Library import

In [ ]:
from peptdeep.pretrained_models import ModelManager
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
model_mgr = ModelManager()

# Load installed models
model_mgr.load_installed_models()

c:\Python310\lib\site-packages\peptdeep\model\model_interface.py:731: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(stream, map_location=self.device), strict=Fals

# Notebook to show that MS2 model from AlphaPeptDeep can not be exploited

In [3]:
data_path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/data"
df = pd.read_excel(f'{data_path}/Liste des peptides unique.xlsx')
df["sequence"] = df["Peptide sequences"]
df["random_number"] = np.random.choice((0, 1), df.shape[0])
df.drop(columns=["Peptide sequences"], inplace=True)
# Force to have those two columns to train => put same value to freeze their contribution
df["mods"] = ''
df["mod_sites"] = ''
df

,sequence,random_number,mods,mod_sites
0,GGVNDNFQGVLQNVR,1,,
1,FVFGTTPEDILR,1,,
2,VDVIPVNLPGEHGQR,1,,
3,STTPDITGYR,0,,
4,SYTITGLQPGTDYK,1,,
...,...,...,...,...
208,GQYCYELDEK,0,,
209,FEDGVLDPDYPR,0,,
210,DWHGVPGQVDAAMAGR,0,,
211,SIAQYWLGCPAPGHL,0,,


In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
pretrained_model = model_mgr.ms2_model
pretrained_model.target_column_to_train = 'random_number'
pretrained_model.train(train_df, epoch=100)

TypeError: pDeepModel.train() missing 1 required positional argument: 'fragment_intensity_df'

In [ ]:
pretrained_model.target_column_to_predict = "random_number_pred"
pretrained_model.predict(test_df)
test_df

,sequence,random_number,mods,mod_sites,nAA,random_number_pred
0,AEIEYLEK,0,,,8,0.115629
1,IVIEYVDR,0,,,8,0.413895
2,EQLTPLIK,0,,,8,0.131349
3,EIGELYLPK,1,,,9,0.687210
4,QFPILLDFK,0,,,9,0.790223
5,CFLAFTQTK,0,,,9,0.347372
6,ILGPLSYSK,0,,,9,0.466691
7,GQTLLAVAK,1,,,9,0.012329
8,TYLPAVDEK,0,,,9,0.071369
9,TFAHYATFR,0,,,9,0.492311


In [ ]:
print(pretrained_model.model.rt_encoder)

Encoder_26AA_Mod_CNN_LSTM_AttnSum(
  (mod_nn): Mod_Embedding_FixFirstK(
    (nn): Linear(in_features=103, out_features=2, bias=False)
  )
  (input_cnn): SeqCNN(
    (cnn_short): Conv1d(35, 35, kernel_size=(3,), stride=(1,), padding=(1,))
    (cnn_medium): Conv1d(35, 35, kernel_size=(5,), stride=(1,), padding=(2,))
    (cnn_long): Conv1d(35, 35, kernel_size=(7,), stride=(1,), padding=(3,))
  )
  (hidden_nn): SeqLSTM(
    (rnn): LSTM(140, 128, num_layers=2, batch_first=True, bidirectional=True)
  )
  (attn_sum): SeqAttentionSum(
    (attn): Sequential(
      (0): Linear(in_features=256, out_features=1, bias=False)
      (1): Softmax(dim=1)
    )
  )
)


In [ ]:
dir(pretrained_model.model.rt_encoder)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_se

In [ ]:
print(pretrained_model.model)

Model_RT_LSTM_CNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (rt_encoder): Encoder_26AA_Mod_CNN_LSTM_AttnSum(
    (mod_nn): Mod_Embedding_FixFirstK(
      (nn): Linear(in_features=103, out_features=2, bias=False)
    )
    (input_cnn): SeqCNN(
      (cnn_short): Conv1d(35, 35, kernel_size=(3,), stride=(1,), padding=(1,))
      (cnn_medium): Conv1d(35, 35, kernel_size=(5,), stride=(1,), padding=(2,))
      (cnn_long): Conv1d(35, 35, kernel_size=(7,), stride=(1,), padding=(3,))
    )
    (hidden_nn): SeqLSTM(
      (rnn): LSTM(140, 128, num_layers=2, batch_first=True, bidirectional=True)
    )
    (attn_sum): SeqAttentionSum(
      (attn): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=False)
        (1): Softmax(dim=1)
      )
    )
  )
  (rt_decoder): Decoder_Linear(
    (nn): Sequential(
      (0): Linear(in_features=256, out_features=64, bias=True)
      (1): PReLU(num_parameters=1)
      (2): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)


In [ ]:
dir(pretrained_model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_as_tensor',
 '_check_predict_in_order',
 '_device',
 '_device_ids',
 '_device_type',
 '_fixed_sequence_len',
 '_get_26aa_indice_features',
 '_get_aa_features',
 '_get_aa_indice_features',
 '_get_aa_indice_features_padding_zeros',
 '_get_aa_mod_features',
 '_get_features_from_batch_df',
 '_get_lr_schedule_with_warmup',
 '_get_mod_features',
 '_get_targets_from_batch_df',
 '_init_for_training',
 '_load_model_from_pytorchfile',
 '_load_model_from_stream',
 '_load_model_from_zipfile',
 '_min_pred_value',
 '_model_to_device',
 '_pad_zeros_if_fixed_len',
 '_predict_one_batch',
 '_prepare_predict_data_df',
 '_prepare_train_dat